<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2024TeamC/blob/main/Datenbank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Kursarbeit Datenmanagement und -Archivierung WS 23/24**

Master-Code Name-Projekt


**Erstellung der Datenbank**


*   Datenbankinitialisierung:



> Importing Libraries

In [ ]:
import pandas as pd
from functools import reduce
import sqlite3
from sqlite3 import Error
import csv
import requests

> Verbindung zu Googledrive herstellen (Ablageort der CSV-Dateien)

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
material_path = "/content/drive/My Drive/csv_data"

In [ ]:
!ls "/content/drive/My Drive/csv_data"

> Verbindung zu SQLite herstellen und Datebankinitialisierung im Memory


In [ ]:
# SQLite connection
# Create a SQLite database in memory from python

def create_connection_memory():
    """ create a database connection to a database that resides
        in the memory
    """
    conn = None;
    try:
       #Establishing the connection
        conn = sqlite3.connect(':memory:')
        return conn
        print(sqlite3.version)
    except Error as e:
        print("Error while connecting to sqlite", e)
    # finally:
    #     if conn:
    #         conn.close()

In [ ]:
# Creating a temporary empty database in the memory.
conn = create_connection_memory()
# Creating a cursor object using the cursor() method
cur = conn.cursor()
print("Successfully Connected to SQLite")


In [ ]:
def create_connection_local(local_path):
  conn = None;
  try:
    #Establishing the connection
    conn = sqlite3.connect(local_path+'/project_public.db')
    return conn
    print(sqlite3.version)
  except Error as e:
    print("Error while connecting to sqlite", e)
conn = create_connection_local('/content/drive/My Drive/Datenmanagement_und_Archivierung_im_Umfeld_der_Forschung')
# Creating a cursor object using the cursor() method
cur = conn.cursor()
print("Successfully Connected to SQLite Public Data Warehouse")


> Quelldatenimport

In [ ]:
# Load breast cancer CSVs
procedures_bc = pd.read_csv('/content/drive/My Drive/csv_data/breast_cancer/procedures.csv', sep=",")
encounters_bc = pd.read_csv('/content/drive/My Drive/csv_data/breast_cancer/encounters.csv', sep=",")
immunizations_bc = pd.read_csv('/content/drive/My Drive/csv_data/breast_cancer/immunizations.csv', sep=",")
medications_bc = pd.read_csv('/content/drive/My Drive/csv_data/breast_cancer/medications.csv', sep=",")
observations_bc = pd.read_csv('/content/drive/My Drive/csv_data/breast_cancer/observations.csv', sep=",")
patients_bc = pd.read_csv('/content/drive/My Drive/csv_data/breast_cancer/patients.csv', sep=",")

In [ ]:
#Delete Tables in case they exist
cur.execute("DROP TABLE IF EXISTS procedures_bc")
cur.execute("DROP TABLE IF EXISTS encounters_bc")
cur.execute("DROP TABLE IF EXISTS immunizations_bc")
cur.execute("DROP TABLE IF EXISTS medications_bc")
cur.execute("DROP TABLE IF EXISTS observations_bc")
cur.execute("DROP TABLE IF EXISTS patients_bc")
cur.execute("DROP TABLE IF EXISTS facts_table")

In [ ]:
# Create tables
sql_create_source_data = requests.get('https://raw.githubusercontent.com/Fuenfgeld/DMA2024TeamC/main/Code/create_statements_db.sql').text
cur.executescript(sql_create_source_data)
conn.commit()
print("Successfully created tables in the database")

In [ ]:
#Insert data into table
procedures_bc.to_sql('procedures_bc', conn, if_exists='append', index=False)
encounters_bc.to_sql('encounters_bc', conn, if_exists='append', index=False)
immunizations_bc.to_sql('immunizations_bc', conn, if_exists='append', index=False)
medications_bc.to_sql('medications_bc', conn, if_exists='append', index=False)
observations_bc.to_sql('observations_bc', conn, if_exists='append', index=False)
patients_bc.to_sql('patients_bc', conn, if_exists='append', index=False)

In [ ]:
#Select data
# show table: retrieve all the column of each records and 4 rows
cur.execute("SELECT * FROM patients_bc")
records = cur.fetchall()
# show 4 rows
for row in records[:4]:
  print(row)

In [ ]:
# List of columns in a table
cur.execute ("select * from procedures_bc")
col_names = cur.description
for row in col_names:
  print(row[0])
